In [1]:

import os
#choose "cpu" or "gpu"
os.environ['THEANO FLAGS']= 'device=gpu'

#following line is useful for fast compilation while using Cpu
#os.environ['THEANO FLAGS']= 'mode=FAST_COMPILE'

#following line is useful for fast compilation while using Gpu
os.environ['THEANO FLAGS']= 'optimizer=fast_compile'

In [2]:
# Implementation of a simple character RNN (using LSTM units), based on:
# https://github.com/karpathy/char-rnn
import numpy as np

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed

text = open('shakespeare.txt', 'r').read()
char_to_idx = {ch: i for (i, ch) in enumerate(sorted(list(set(text))))}
idx_to_char = {i: ch for (ch, i) in char_to_idx.items()}
vocab_size = len(char_to_idx)

print 'Working on %d characters (%d unique)' % (len(text), vocab_size)

SEQ_LEN = 64
BATCH_SIZE = 16
BATCH_CHARS = len(text) / BATCH_SIZE
LSTM_SIZE = 128
LAYERS = 3

Using Theano backend.
Using gpu device 0: GeForce GTX 480 (CNMeM is disabled, cuDNN not available)


Working on 1115395 characters (65 unique)


In [3]:
# For training, each subsequent example for a given batch index should be a
# consecutive portion of the text.  To achieve this, each batch index operates
# over a disjoint section of the input text.
def read_batches(text):
    T = np.asarray([char_to_idx[c] for c in text], dtype=np.int32)
    X = np.zeros((BATCH_SIZE, SEQ_LEN, vocab_size))
    Y = np.zeros((BATCH_SIZE, SEQ_LEN, vocab_size))

    for i in range(0, BATCH_CHARS - SEQ_LEN - 1, SEQ_LEN):
        X[:] = 0
        Y[:] = 0
        for batch_idx in range(BATCH_SIZE):
            start = batch_idx * BATCH_CHARS + i
            for j in range(SEQ_LEN):
                X[batch_idx, j, T[start+j]] = 1
                Y[batch_idx, j, T[start+j+1]] = 1

        yield X, Y

In [4]:
def build_model(infer):
    if infer:
        batch_size = seq_len = 1
    else:
        batch_size = BATCH_SIZE
        seq_len = SEQ_LEN
    model = Sequential()
    model.add(LSTM(LSTM_SIZE,
                   return_sequences=True,
                   batch_input_shape=(batch_size, seq_len, vocab_size),
                   stateful=True))

    model.add(Dropout(0.2))
    for l in range(LAYERS - 1):
        model.add(LSTM(LSTM_SIZE, return_sequences=True, stateful=True))
        model.add(Dropout(0.2))

    model.add(TimeDistributed(Dense(vocab_size)))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adagrad')
    return model

In [5]:
def sample(epoch, sample_chars=256, primer_text='And the '):
    test_model.reset_states()
    test_model.load_weights('/tmp/keras_char_rnn.%d.h5' % epoch)
    sampled = [char_to_idx[c] for c in primer_text]

    for c in primer_text:
        batch = np.zeros((1, 1, vocab_size))
        batch[0, 0, char_to_idx[c]] = 1
        test_model.predict_on_batch(batch)

    for i in range(sample_chars):
        batch = np.zeros((1, 1, vocab_size))
        batch[0, 0, sampled[-1]] = 1
        softmax = test_model.predict_on_batch(batch)[0].ravel()
        sample = np.random.choice(range(vocab_size), p=softmax)
        sampled.append(sample)

    print ''.join([idx_to_char[c] for c in sampled])


In [6]:
print 'Building model.'
training_model = build_model(infer=False)
test_model = build_model(infer=True)
print '... done'

for epoch in range(300):
    for i, (x, y) in enumerate(read_batches(text)):
        loss = training_model.train_on_batch(x, y)
        print epoch, i, loss

        if i % 1000 == 0:
            training_model.save_weights('/tmp/keras_char_rnn.%d.h5' % epoch,
                                        overwrite=True)
            sample(epoch)

Building model.
... done
0 0 4.18618583679
And the -r&xe hyUhf:alnut
TDWtMt?ejWF z  eff$LHLWSxDBfJQxP?VbX
mUohCeT:taX!HJrwQiZvsde:StxMRpw!Wnht,lm.-nNRf.JUH 3t:Lrj:$HfGfb yzNy iKOL?O'b?SazeakQYUxlrZT$JtyOTvsltnstfGlJz:$ocL'dzAJJ?gUGj v;WPqrBRu;l!A ;ybLdu3EOLRrMV-h'I,':xey'KH  
?vJ$RokpK:LeXzxWtpx;?hBlfWwbva
0 1 3.83594369888
0 2 3.51570105553
0 3 3.51976680756
0 4 3.3616642952
0 5 3.32989048958
0 6 3.30304908752
0 7 3.23021173477
0 8 3.19847154617
0 9 3.38184261322
0 10 3.32916045189
0 11 3.31571722031
0 12 3.32291722298
0 13 3.32061100006
0 14 3.30937361717
0 15 3.37354183197
0 16 3.41029453278
0 17 3.40133285522
0 18 3.33449268341
0 19 3.31142663956
0 20 3.32329893112
0 21 3.29683804512
0 22 3.36400842667
0 23 3.39225721359
0 24 3.32646632195
0 25 3.35317468643
0 26 3.33649253845
0 27 3.31867790222
0 28 3.29365754128
0 29 3.33353543282
0 30 3.31903886795
0 31 3.36978435516
0 32 3.36053752899
0 33 3.3115735054
0 34 3.35204124451
0 35 3.31494140625
0 36 3.21749305725
0 37 3.38516783714


KeyboardInterrupt: 